---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
        
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    uni_towns = pd.read_table('university_towns.txt', header=None)
    uni_towns.columns = ['RegionName'] 
    uni_towns['State'] = uni_towns['RegionName'][uni_towns.RegionName.str.contains(r"\[edit\]")]
    uni_towns.fillna(method='ffill', inplace=True)
    uni_towns = uni_towns[~uni_towns.RegionName.str.contains(r"\[edit\]")]
    # cleanup
    uni_towns['State']=uni_towns['State'].str.replace(r"\[.*\]","")
    uni_towns['RegionName']=uni_towns['RegionName'].str.replace(r"\(.*","")
    uni_towns['RegionName']=uni_towns['RegionName'].str.replace(r"\[.*\]","")
    uni_towns['RegionName'] = uni_towns.RegionName.str.strip()
    
    
    return uni_towns.reset_index(drop=True)[['State', 'RegionName']]

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    recession_data = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows = 219)
    recession_data=recession_data[['1999q4',9926.1]]
    recession_data.columns = ['quarter','gdp']

    # find the start of recession periods
    recession_ix = []
    for i in range(0, len(recession_data.gdp.values)-2):
        if recession_data.gdp[i+2] < recession_data.gdp[i+1] < recession_data.gdp[i]:
            recession_ix.append(recession_data.quarter.values[i])
    return recession_ix[0]
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_data = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows = 219)
    recession_data=recession_data[['1999q4',9926.1]]
    recession_data.columns = ['quarter','gdp']

    # find the start of recession periods
    start = recession_data[recession_data['quarter'] == get_recession_start()].index[0]
    # subset the data from the start of the index
    recession_data = recession_data.iloc[start:].reset_index(drop = True)
    
    #get the recession end
    recession_ix = []
    for i in range(0, len(recession_data.gdp.values)-2):
        if recession_data.gdp[i+2] > recession_data.gdp[i+1] > recession_data.gdp[i]:
            return recession_data.quarter.values[i+2]
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession_data = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows = 219)
    recession_data=recession_data[['1999q4',9926.1]]
    recession_data.columns = ['quarter','gdp']
    start = recession_data[recession_data['quarter'] == get_recession_start()].index[0]
    end = recession_data[recession_data['quarter'] == get_recession_end()].index[0]
    recession_data = recession_data.iloc[start:end].reset_index(drop = True)
    
    return recession_data[recession_data.gdp == recession_data.gdp.min()]['quarter'].values[0]
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data['State'] = housing_data['State'].map(states)
    housing_data.set_index(['State','RegionName'],inplace=True)
    housing_data.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
    housing_data= housing_data.loc[:,'2000-01':]
    housing_data.columns = pd.to_datetime(housing_data.columns)
    housing_data = housing_data.resample('Q', axis=1).mean()
    housing_data = housing_data.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter))
    return housing_data
convert_housing_data_to_quarters().shape

(10730, 67)

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    towns = set(get_list_of_university_towns()['RegionName'])
    
    data = convert_housing_data_to_quarters()
    data = data.loc[:,get_recession_start():get_recession_end()].reset_index()
    
    
    # generate a price change column 
    data['price_ratio'] = (data['2008q3']).div(data['2009q2'])
    
    # drop na values
    data = data[data.price_ratio.notnull()]
    
    # create a column indicating if the region name is in a university town
    data['is_uni_town'] = np.where(data.RegionName.isin(towns), 1, 0)
    
    # run the t test
    t_test_result = ttest_ind(
            data[data['is_uni_town']==1].loc[:,'price_ratio'], 
            data[data['is_uni_town']==0].loc[:,'price_ratio'])
    
    p = t_test_result[1]
    
    return (True,p,'university town')
run_ttest()

(True, 0.00019452075242294589, 'university town')